In [127]:
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import division

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',200)

%matplotlib inline

plt.style.use('ggplot')

# Import/cleaning

In [75]:
licenses = pd.read_csv('./../working-data/bzn-biz-licenses.csv')

In [76]:
# Address concatenation
# TODO - shift to parser file

def str_not_nan(value):
    if pd.isnull(value):
        return ""
    else:
        return str(value)

addr_keys = ['street_num', 'street_dir_code', 'street_name', 'street_suffix']
licenses['address'] = licenses[addr_keys].apply(lambda x: ' '.join(x.map(str_not_nan)), axis=1)

unit_keys = ['street_post_qual', 'street_apt_num']
licenses['unit'] = licenses[unit_keys].apply(lambda x: ' '.join(x.map(str_not_nan)), axis=1)

street_keys = ['street_dir_code','street_name']
licenses['street'] = licenses[street_keys].apply(lambda x: ' '.join(x.map(str_not_nan)), axis=1)

In [77]:
print licenses.columns
licenses.head(2)

Index([u'biz_name', u'license_num', u'control_num', u'resp_dept_code',
       u'resp_dept_descr', u'class_code', u'class_descr', u'fiscal_year',
       u'license_status', u'code_descr', u'status_date', u'application_date',
       u'issue_date', u'licen_exp_date', u'renew_date', u'location_id',
       u'street_num', u'street_pre_qual', u'street_dir_code', u'street_name',
       u'street_suffix', u'street_post_qual', u'street_apt_num',
       u'class_desc_cat', u'class_desc_detail', u'address', u'unit',
       u'street'],
      dtype='object')


,biz_name,license_num,control_num,resp_dept_code,resp_dept_descr,class_code,class_descr,fiscal_year,license_status,code_descr,status_date,application_date,issue_date,licen_exp_date,renew_date,location_id,street_num,street_pre_qual,street_dir_code,street_name,street_suffix,street_post_qual,street_apt_num,class_desc_cat,class_desc_detail,address,unit,street
0,... AND SEW ON,52085,25732,BL,BUSINESS LICENSE,SER 28,SERVICE - SEWING/ALTERATIONS,15,IN,INACTIVE,2015-12-16,2015-01-22,2015-08-02,2015-12-31,NaN,52760,601,NaN,NaN,NIKLES,DR,NaN,NaN,SERVICE,SEWING/ALTERATIONS,601 NIKLES DR,,NIKLES
1,... AND SEW ON,53948,25732,BL,BUSINESS LICENSE,RET 38,RETAIL - OTHER,16,AC,ACTIVE,2016-03-02,2015-12-16,2016-03-02,2016-12-31,NaN,52760,601,NaN,NaN,NIKLES,DR,NaN,NaN,RETAIL,OTHER,601 NIKLES DR,,NIKLES


In [78]:
# Filter to active/renewed licesnses & interest columns
lic = licenses[licenses['code_descr'].isin(['RENEWED', 'ACTIVE'])]

int_cols = ['street', 'street_num', 'address','unit','location_id',
            'biz_name','class_desc_cat','class_desc_detail',
            'issue_date', 'fiscal_year'
            ]
lic = lic[int_cols]

In [82]:
# Filter to businesses on downtown streets

# Address ranges in B-3 zoning district, hand-compiled
downtown_addr = [
    {'street': 'W MAIN', 'min': 0, 'max': 499},
    {'street': 'W BABCOCK', 'min': 0, 'max': 399 },
    {'street': 'W MENDENHALL', 'min': 0, 'max': 299},
    {'street': 'W LAMME', 'min': 0, 'max': 209},
    {'street': 'W BEALL', 'min': 0, 'max': 199},
    
    {'street': 'E MAIN', 'min': 0, 'max': 799},
    {'street': 'E BABCOCK', 'min': 0, 'max': 599},
    {'street': 'E OLIVE', 'min': 0, 'max': 199},
    {'street': 'E MENDENHALL', 'min': 0, 'max': 599},
    {'street': 'E LAMME', 'min': 0, 'max': 199},
    {'street': 'E BEALL', 'min': 0, 'max': 99},
    
    {'street': 'N 3RD', 'min': 0, 'max': 99},
    {'street': 'N GRAND', 'min': 0, 'max': 312},
    {'street': 'N WILLSON', 'min': 0, 'max': 399},
    {'street': 'N TRACY', 'min': 0, 'max': 299},
    {'street': 'N BLACK', 'min': 0, 'max': 301}, # Includes one extra house
    {'street': 'N BOZEMAN', 'min': 0, 'max': 299}, # Includes one extra house, 204 N Bozeman
    {'street': 'N ROUSE', 'min': 0, 'max': 199},
    {'street': 'N CHURCH', 'min': 0, 'max': 99},
    {'street': 'N WALLACE', 'min': 0, 'max': 99},
    
    {'street': 'S 3RD', 'min': 0, 'max': 99},
    {'street': 'S GRAND', 'min': 0, 'max': 99},
    {'street': 'S WILLSON', 'min': 0, 'max': 213},
    {'street': 'S TRACY', 'min': 0, 'max': 212}, # Includes one extra house
    {'street': 'S BLACK', 'min': 0, 'max': 214},
    {'street': 'S BOZEMAN', 'min': 0, 'max': 209}, # Includes extra, 206 S Bozeman
    {'street': 'S ROUSE', 'min': 0, 'max': 99},
    {'street': 'S CHURCH', 'min': 0, 'max': 112},
    {'street': 'S WALLACE', 'min': 0, 'max': 215}, # Includes extras, 115, 117, 121
]

# expects district as array of {'street', 'min', 'max'} specifying address range along specific street
def get_rows_for_address_range(df, addr_range):
    street = addr_range['street']
    addr_min = addr_range['min']
    addr_max = addr_range['max']
    return df[(df['street'] == street) & (df['street_num'] <= addr_max) & (df['street_num'] >= addr_min)]

def get_rows_for_district(df, district):
    output = df[0:0]
    for addr_range in district:
        partial = get_rows_for_address_range(df, addr_range)
        output = pd.concat([output, partial])
    return output

downtown = get_rows_for_district(df, downtown_addr)

# Aggregations

In [79]:
aggregate = {
    'issue_date': {'first_date': min, 'last_date': max, 'renewals': len},
    'class_desc_cat': min,
    'class_desc_detail': min
}

by_location = lic.groupby(['street', 'street_num', 'address', 'unit', 'location_id', 'biz_name']).agg(aggregate)

In [81]:
# by_location

In [76]:
by_location.to_excel('./../working-data/main-street-biz-by-location.xlsx')

In [26]:
by_loc_and_year = pd.pivot_table(lic,
                                values='issue_date',
                                index=['street', 'street_num', 'address','unit','location_id','biz_name','class_desc_cat','class_desc_detail'],
                                columns='fiscal_year',
                                aggfunc=min)


In [72]:
# by_loc_and_year

In [29]:
all_by_year = pd.pivot_table(lic,
                                values='issue_date',
                                index=['street', 'street_num', 'address','unit','location_id', 'biz_name','class_desc_cat','class_desc_detail'],
                                columns='fiscal_year',
                                aggfunc=min)

In [77]:
by_loc_and_year.to_excel('./../working-data/main-street-biz-by-loc-and-year.xlsx')

In [99]:
by_loc_and_year.to_csv('./../working-data/main-street-biz-by-loc-and-year.csv')

In [30]:
all_by_year

fiscal_year                                                                                                                                                                    2   \
street       street_num address                unit        location_id biz_name                                         class_desc_cat class_desc_detail                            
 ACCOLA      207        207  ACCOLA DR                     39360       CRYSTAL RIVER CUSTOM                             SERVICE        INTERIOR DESIGN                       None   
                                                                       CRYSTAL SEA SOUND                                SERVICE        OTHER                                 None   
                                                                       TREE OF LIFE                                     SERVICE        OTHER                                 None   
                                                                       UNLOCKED DESIGNS                                 SERVICE        OTHER                                 None   
             210        210  ACCOLA DR                     39370       DUMBROVSKA, RADKA                                SERVICE        MARKETING & ADVERTISING               None   
             215        215  ACCOLA DR                     39260       ECO MONTANA                                      SERVICE        OTHER                                 None   
                                                                       SDI                                              CONTRACTORS    HANDY PERSONS                         None   
             216        216  ACCOLA DR                     39250       WHOLE LIFE FAMILIES                              SERVICE        CONSULTING                            None   
             303        303  ACCOLA DR                     39270       PIK UP!                                          SERVICE        CLEANING SERVICE                      None   
 ALDER CREEK 1096       1096  ALDER CREEK DR               172720      COPPER HAMMER CONSTRUCTION                       CONTRACTORS    GENERAL                               None   
 ALEXANDER   4433       4433  ALEXANDER ST                 116240      CAPSTONE CLEANING                                SERVICE        CLEANING SERVICE                      None   
             4442       4442  ALEXANDER ST                 116000      JACKPINE INC DBA OXIFRESH OF MONTANA             SERVICE        CLEANING SERVICE                      None   
                                                                       SINGLETRACK INC                                  RETAIL         OTHER                                 None   
             4475       4475  ALEXANDER ST                 116220      KIRCHHOFF CONSTRUCTION                           CONTRACTORS    GENERAL                               None   
             4512       4512  ALEXANDER ST                 116040      GREEN PAWS LLC                                   SERVICE        OTHER                                 None   
                                                                       SANFORD RED, LLC                                 RENTALS        VACATION RENTALS                      None   
             4515       4515  ALEXANDER ST                 116190      PRIME ROOFING                                    CONTRACTORS    ROOFING                               None   
             4531       4531  ALEXANDER ST                 116180      SPRUNG~ ARTWORK WITH A TWIST                     SERVICE        OTHER                                 None   
             4610       4610  ALEXANDER ST                 116100      DEFRANCE CONSTRUCTION                            CONTRACTORS    GENERAL                         2002-01-16   
 ALLISON     2706       2706  ALLISON CT                   99420       ALLISON WONDERLAND                               SERVICE        CHILDCARE                             None   
             2717 

In [31]:
all_by_year.to_csv('./../working-data/bzn-biz-lic-by-year.csv')

In [83]:
len(downtown)

4745

In [84]:
downtown_by_year = pd.pivot_table(downtown,
                                values='issue_date',
                                index=['street', 'street_num', 'address','unit','location_id', 'biz_name','class_desc_cat','class_desc_detail'],
                                columns='fiscal_year',
                                aggfunc=min)

In [85]:
downtown_by_year

fiscal_year                                                                                                                                                       2   \
street       street_num address             unit       location_id biz_name                                class_desc_cat class_desc_detail                            
E BABCOCK    106        106 E BABCOCK ST               5200        BOUNTIFUL TABLE CUSTOM CATERNG          FOOD SERVICE   CATERING                        2002-01-03   
                                            UNIT C     211570      ENGINEERING INC DBA SANDERSON STEWART   SERVICE        OTHER                                 None   
             234        234 E BABCOCK ST               2500        AFFORDABLE ACCOUNTING INC               SERVICE        FINANCIAL/INVESTMENT            2002-01-16   
                                                                   BODIES IN MOTION THERAPEUTICS LLC       SERVICE        HEALTH CARE                           None   
                                                                   MOUNTAIN YOGA                           SERVICE        DANCE STUDIO                          None   
                                                                   PERFECT 10 NAILS                        RETAIL         BEAUTY & HEALTH                 2002-01-02   
                                                                   RELAXING REMEDIES MASSAGE               SERVICE        MASSAGE THERAPY                 2002-02-22   
                                                                   SALON SIMPATICO                         SERVICE        HEALTH & BEAUTY                       None   
                                                                   SEWING AND ALTERATIONS                  SERVICE        SEWING/ALTERATIONS              2002-02-04   
                                                                   SUN & MOON MONTANA                      SERVICE        MASSAGE THERAPY                       None   
             316        316 E BABCOCK ST               2540        NOT SO NU2U! THRIFT OUTLET              RETAIL         CLOTHING AND/OR SHOES                 None   
             414        414 E BABCOCK ST               2220        BOZEMAN BOWL, THE                       FOOD SERVICE   RESTAURANT                      2002-02-15   
             520        520 E BABCOCK ST               1740        INDIRA SPA & WELLNESS                   SERVICE        HEALTH & BEAUTY                       None   
                                                                   JOHN RIMI CUSTOM FINISHES               CONTRACTORS    PAINTING                              None   
             533        533 E BABCOCK ST    UNITS 1-10 870         THE PELLERIN COMPANY LLC                SERVICE        FINANCIAL/INVESTMENT                  None   
             541        541 E BABCOCK ST               850         EAST BABCOCK LLC                        RENTALS        VACATION RENTALS                      None   
                                                                   SERVICE ADVENTURES INT'L LLC            SERVICE        OTHER                                 None   
             548        548 E BABCOCK ST               880         FRANK'S GOURMET GRILL & DELI            FOOD SERVICE   CATERING                        2002-05-31   
                                                                   SWEET THINGS                            FOOD SERVICE   CATERING                              None   
E BEALL      7          7 E BEALL ST                   27050       OFF THE BEATEN PATH,LLC                 RETAIL         TRAVEL AGENCY                   2002-01-24   
E LAMME      108        108 E LAMME ST                 96690       KOKOPELLI ARTS                          SERVICE        CLEANING SERVICE                      None   
             114        114 E LAMME ST                 30790       LAST BEST CHEFS                         SERVICE        OTHER         

In [87]:
downtown_by_year.to_csv('./../working-data/downtown-biz-by-year.csv')
downtown_by_year.to_csv('./../app/data/downtown-biz-by-year.csv')

# How has business composition changed?

## Downtown

In [115]:
type_by_year = pd.pivot_table(downtown,
                values='class_desc_cat',
                index=['biz_name'],
                columns='fiscal_year',
                aggfunc=min)

In [108]:
type_by_year.head(1)

fiscal_year,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
biz_name,,,,,,,,,,,,,,,
14 NORTH,None,None,None,None,None,None,None,None,None,None,None,FOOD SERVICE,FOOD SERVICE,FOOD SERVICE,FOOD SERVICE


In [122]:
type_counts = type_by_year.apply(pd.Series.value_counts)
type_counts.fillna(0, inplace=True)

In [134]:
type_counts.to_csv('./../working-data/cat-counts-by-year.csv')

## Main Street

In [137]:
main_street = downtown[downtown['street'].str.contains('MAIN')]

In [138]:
len(main_street)

2831

In [139]:
main_street_by_year = pd.pivot_table(
    downtown,
    values='class_desc_cat',
    index=['biz_name'],
    columns='fiscal_year',
    aggfunc=min
)
main_street_by_year = main_street_by_year.apply(pd.Series.value_counts)
main_street_by_year.fillna(0, inplace=True)

In [141]:
main_street_by_year.to_csv('./../working-data/ms-cat-counts-by-year.csv')

# Future questions

- How are liquor licenses changing?